In [13]:
#installations
!pip install torchaudio
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
ERROR: Operation cancelled by user


In [2]:
#imports
import os
import music21
import pretty_midi
from music21 import midi
import pandas as pd
from IPython.display import Audio
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import statistics
import math, random, time
from IPython.display import Audio
from google.colab import drive
from google.colab import files
import torchaudio
from collections import Counter
import numpy as np
import torch
import torch.nn as nn
import scipy
import shutil

In [3]:
#mount drive, establish data directory
from google.colab import drive
drive.mount('/content/drive')
data_directory = '/content/drive/MyDrive/maestro-v2.0.0/'

Mounted at /content/drive


In [4]:
# define midi reading function
def readMidi(filepath):
  mf = midi.MidiFile()
  mf.open(filepath)
  mf.read()
  mf.close()
  return mf

In [5]:
# Need a token dictionary and an inverse token dictionary
token_dict = {}
inverse_token_dict = {}
for i in range(128):
  token_dict[i] = i
  inverse_token_dict[i] = i

token_dict['<rest>'] = 128
token_dict['<bos>'] = 129
token_dict['<eos>'] = 130

inverse_token_dict[128] = '<rest>'
inverse_token_dict[129] = '<bos>'
inverse_token_dict[130] = '<eos>'

In [18]:
import os
import shutil
from tqdm import tqdm
import pretty_midi
import numpy as np
import torch
from torch.utils.data import Dataset

class MaestroDataset(Dataset):
    def __init__(self, split, token_dict, directory='/content/drive/MyDrive/maestro-v2.0.0/'):
        import pandas as pd

        self.split = split
        self.token_dict = token_dict
        self.directory = directory.rstrip('/') + '/'
        # read the CSV and filter to our split
        df = pd.read_csv(os.path.join(self.directory, 'maestro-v2.0.0.csv'))
        self.df = df[df['split'] == split].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def moveFilesToSplitDir(self):
        src_root = self.directory
        dst_root = os.path.join(self.directory, self.split)
        os.makedirs(dst_root, exist_ok=True)

        for fname in self.df['midi_filename']:
            src = os.path.join(src_root, fname)
            if not os.path.isfile(src):
                print(f" ⚠️  Warning: MIDI file not found: {src}")
                continue
            dst = os.path.join(dst_root, os.path.basename(fname))
            shutil.move(src, dst)

    def create_token_sequences(self,
                               pm: pretty_midi.PrettyMIDI,
                               fs: int = 16,
                               seq_len: int = 500,
                               add_bos_eos: bool = True):
        pianoroll = pm.get_piano_roll(fs=fs)
        pianoroll = (pianoroll > 0).astype(int)

        num_frames = pianoroll.shape[1]
        seqs = []
        for start in range(0, num_frames, seq_len):
            end = start + seq_len
            if end > num_frames:
                break
            block = pianoroll[:, start:end]
            # skip mostly-silent windows
            if block.sum() == 0:
                continue

            seq = []
            if add_bos_eos:
                seq.append(self.token_dict['<bos>'])
            for t in range(seq_len):
                notes = np.nonzero(block[:, t])[0]
                if len(notes):
                    seq.append(int(notes[0]))
                else:
                    seq.append(self.token_dict['<rest>'])
            if add_bos_eos:
                seq.append(self.token_dict['<eos>'])
            seqs.append(seq)
        return seqs

    def augment_idxs(self, seq):
        augmented = []
        for shift in range(-12, +13):
            out = []
            for idx in seq:
                # only shift real MIDI notes; leave special tokens alone
                if 0 <= idx < 128:
                    n = idx + shift
                    n = max(0, min(127, n))
                    out.append(n)
                else:
                    out.append(idx)
            augmented.append(out)
        return augmented

    def create_all_token_sequences(self,
                                   fs: int = 16,
                                   seq_len: int = 500):
        split_dir = os.path.join(self.directory, self.split)
        all_seqs = []

        for fn in tqdm(os.listdir(split_dir)):
            if not fn.lower().endswith('.midi'):
                continue
            path = os.path.join(split_dir, fn)
            try:
                pm = pretty_midi.PrettyMIDI(path)
            except Exception as e:
                print(f" ⚠️  Skipping {fn}: {e}")
                continue

            seqs = self.create_token_sequences(pm, fs=fs, seq_len=seq_len)
            for s in seqs:
                all_seqs.append(s)
                all_seqs.extend(self.augment_idxs(s))

        return all_seqs

    def tokens_seqs_to_txt(self, seqs, phase):
        split_dir = os.path.join(self.directory, self.split)
        out_path = os.path.join(split_dir, f"{phase}.txt")
        with open(out_path, 'w') as f:
            for seq in seqs:
                f.write(" ".join(map(str, seq)) + "\n")


In [20]:
# build token dict once
token_dict = {i:i for i in range(128)}
token_dict.update({'<rest>':128, '<bos>':129, '<eos>':130})

for phase in ['train', 'test', 'validation']:
    ds = MaestroDataset(phase, token_dict,
                        directory='/content/drive/MyDrive/maestro-v2.0.0/')

    ds.moveFilesToSplitDir()
    seqs = ds.create_all_token_sequences(fs=16, seq_len=500)
    ds.tokens_seqs_to_txt(seqs, phase)

 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1-D6_09_Track09_wav.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2013/ORIG-MIDI_02_7_10_13_Group_MID--AUDIO_11_R3_2013_wav--3.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2011/MIDI-Unprocessed_09_R1_2011_MID--AUDIO_R1-D3_

100%|██████████| 116/116 [01:12<00:00,  1.61it/s]


 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_08_WAV.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2018/MIDI-Unprocessed_Recital13-15_MID--AUDIO_15_R1_2018_wav--1.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2014/MIDI-UNPROCESSED_09-10_R1_2014_MID--AUDIO_09_R1_2014_wav--4.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2009/MIDI-Unprocessed_11_R1_2009_01-05_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_04_WAV.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v

100%|██████████| 56/56 [00:41<00:00,  1.34it/s]


 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2013/ORIG-MIDI_01_7_6_13_Group__MID--AUDIO_03_R1_2013_wav--4.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--3.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--4.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2018/MIDI-Unprocessed_Recital8_MID--AUDIO_08_R1_2018_wav--2.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2018/MIDI-Unprocessed_Recital9-11_MID--AUDIO_10_R1_2018_wav--5.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2013/ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_06_R1_2013_wav--4.midi
 ⚠️  Warning: MIDI file not found: /content/drive/MyDrive/maestro-v2.0.0/2011/MIDI-Unprocessed_16_R1_2011_MID--AUDIO_R1-D6_15_Track15_wav.midi
 ⚠️  Warning: 

100%|██████████| 59/59 [00:45<00:00,  1.30it/s]
